In [123]:
import numpy as np
import math

# Calculate Entropy
def shannonEntropy(data):
    numEntries = len(data)
    labelCounts = {}
    for record in data:
        label = record[-1]
        if label not in labelCounts:
            labelCounts[label] = 1
        else:
            labelCounts[label] += 1
    shannonEnt = 0
    
    for key in labelCounts:
        prob = labelCounts[key]/(numEntries*1.0)
        #print('prob %f' %prob)
        shannonEnt -= prob * math.log(prob, 2)
    return shannonEnt  

In [150]:
dataSet = np.array([[0,0,'yes'],[0,1,'yes'],[1,0,'yes'],[1,0,'yes']])
print(shannonEntropy(dataSet))

0.0


In [151]:
dataSet = np.array([[0,0,'yes'],[0,1,'no'],[1,0,'yes'],[1,1,'yes']])
print(shannonEntropy(dataSet))

0.811278124459


In [152]:
dataSet = np.array([[0,0,'yes'],[0,1,'no'],[1,0,'no'],[1,1,'yes']])
print(shannonEntropy(dataSet))

1.0


### Entropy value indicates impurity of data. 熵的值越大，则数据纯度越低；如果熵的值是最大值1，则数据的纯度最低，即每种label的数据均匀分布。

In [161]:
# Split the data by a feature
def splitData(data,featureNameInx,featureValue):
    selectedData = []
    for record in data:

        if str(record[featureNameInx]) == str(featureValue):
            # Remove record[featureNameInx] from the record
            reducedFeatVec = list(record[:featureNameInx]) + list(record[featureNameInx+1:])
            selectedData.append(reducedFeatVec)

    return selectedData

In [154]:
print((splitData(dataSet,0,1)))

[['0', 'no'], ['1', 'yes']]


In [215]:
# Iterate all features and split the data in such a way that maximun entropy/info gain is achieved.
def bestFeatureSplit(data):
    baseEntropy = shannonEntropy(data)
    bestInfoGain = 0.0
    bestFeature = -1
    
    # The last 'feature' is actually the label/class
    numFeatures = len(data[0]) - 1
    for i in range(numFeatures):
        print(i)
        # Get the distinct value set for the ith column
        uniqueVals = set([record[i] for record in data])
        totalNewEntropy = 0.0
        for v in uniqueVals:
            subData = splitData(data, i, v)
            prob = len(subData)/(len(data)*1.0)
            totalNewEntropy += prob*shannonEntropy(subData)
            
        infoGain = baseEntropy - totalNewEntropy
        print(infoGain)
        if infoGain > bestInfoGain :
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature


In [155]:
dataSet = np.array([[0,0,'yes'],[0,1,'no'],[1,0,'yes'],[1,1,'no']])
print(bestFeatureSplit(dataSet))

1


In [174]:
# Build the decision tree in Recursion
def createTree(data, featureNames):
    classList = [record[-1] for record in data]
    # Return if all data have the same label
    uniqueLabels = set(classList)
    if len(uniqueLabels) == 1:
        print('Return if all data have the same label')
        return classList[0]
    
    # Return if there are no more feature columns
    if len(data[0]) == 1:
        print('Return if there are no more feature columns')
        return majorityCnt(classList)
    
    # Select the feature that best split the data 
    bestFeatureInx = bestFeatureSplit(data)
    bestFeatName = featureNames[bestFeatureInx]
    print('Split by feature name '+bestFeatName)
    myTree = {bestFeatName:{}}
    del(featureNames[bestFeatureInx])
    
    # Recursively split the data within each splited set
    featValues = [record[bestFeatureInx] for record in data]
    uniqueVals = set(featValues)
    print(uniqueVals)
    for v in uniqueVals:
        print(v)
        subfeatureNames = featureNames[:]
        myTree[bestFeatName][v] = createTree(splitData(data,bestFeatureInx,v),subfeatureNames)
    
    # Final Decision Tree is a nested dict
    return myTree

In [157]:
def majorityCnt(labelList):
    classCount = {}
    for c in labelList:
        if c not in classCount:
            classCount[c] = 1
        else:
            classCount[c] += 1
            
    sortedClassCount = sorted(classCount.items(), key=lambda d: d[1], reverse=True)
    return sortedClassCount[0][0]

In [177]:
featureNames = ['No Surfacing','Flippers','Fish']
dataSet = np.array([['NoSurf','BigFeet','Yes'],['Surf','SmallFeet','No'],['NoSurf','BigFeet','No'],['NoSurf','SmallFeet','No'],['NoSurf','BigFeet','No']])
tree = createTree(dataSet,featureNames)
print(tree)


Split by feature name Flippers
set(['SmallFeet', 'BigFeet'])
SmallFeet
Return if all data have the same label
BigFeet
Split by feature name No Surfacing
set(['NoSurf'])
NoSurf
Return if there are no more feature columns
{'Flippers': {'SmallFeet': 'No', 'BigFeet': {'No Surfacing': {'NoSurf': 'No'}}}}


### Classifier
### Below is a ID3 implementation. It works well on categorical data.

In [189]:
def decisionTreeClassifer(decisionTree, testData, featureNames):
    firstFeature = decisionTree.keys()[0]
    featureInx = featureNames.index(firstFeature)
    subDict = decisionTree[firstFeature]
    
    for key in subDict.keys():
        if testData[featureInx] == key:
            if type(subDict[key]).__name__ == 'dict':
                classLabel = decisionTreeClassifer(subDict[key], testData, featureNames)
            else:
                classLabel = subDict[key]
    return classLabel 

In [190]:
featureNames = ['No Surfacing','Flippers','Fish']
testData = ['NoSurf','BigFeet','Yes']
print(decisionTreeClassifer(tree, testData, featureNames))

No


In [192]:
import pandas as pd

data = pd.read_csv('Lenses.txt', sep="\t", header=None)
data.columns = ["Age", "Spectacle", "Astigmatic", "Tear","Type"]

data.head()

,Age,Spectacle,Astigmatic,Tear,Type
0,young,myope,no,reduced,no lenses
1,young,myope,no,normal,soft
2,young,myope,yes,reduced,no lenses
3,young,myope,yes,normal,hard
4,young,hyper,no,reduced,no lenses


In [217]:
leneseTree = createTree(data.values,list(data.columns))
print(leneseTree)

0
0.0393965036461
1
0.0395108354236
2
0.377005230011
3
0.548794940695
Split by feature name Tear
set(['reduced', 'normal'])
reduced
Return if all data have the same label
normal
0
0.221251836004
1
0.0954372523106
2
0.770426041486
Split by feature name Astigmatic
set(['yes', 'no'])
yes
0
0.251629167388
1
0.459147917027
Split by feature name Spectacle
set(['hyper', 'myope'])
hyper
0
0.918295834054
Split by feature name Age
set(['pre', 'presbyopic', 'young'])
pre
Return if all data have the same label
presbyopic
Return if all data have the same label
young
Return if all data have the same label
myope
Return if all data have the same label
no
0
0.316689088315
1
0.190874504621
Split by feature name Age
set(['pre', 'presbyopic', 'young'])
pre
Return if all data have the same label
presbyopic
0
1.0
Split by feature name Spectacle
set(['hyper', 'myope'])
hyper
Return if all data have the same label
myope
Return if all data have the same label
young
Return if all data have the same label
{'Tear